`weave` objects controls the output behaviours of a string containing source code.

In [1]:
from dataclasses import dataclass, field
import collections, mistune, IPython, string, jinja2, operator, textwrap, tokenize, trio, toolz.curried as toolz, vdom

In [2]:
try:
    from .strings.markdown import renderer
    from .strings.environment import environment, IPythonDollarTemplate
except:
    from strings.markdown import renderer
    from strings.environment import environment, IPythonDollarTemplate

In [3]:
ip = IPython.get_ipython()

## Statements and Expressions

In Python, `exec` is an statement and `eval` is a expression.  A difference between these function is that `exec` does not return a result and `eval` does return a result.  `exec` accepts any Python statements or expressions while `eval` accepts a subset of Python; assignments are not allowed.. 

`Text` is an `object` that manages the statements and expressions contained within a block of __Markdown__ text.

`Text.data` contains the input source text may be __Markdown__; indented source code is valid __Markdown__ code.  With `Text.markdown` enabled `Text.data` is:

1. Passes through a __Markdown__ parser to extract the

    * block code statements and expressions 
    * inline code expressions
    
    > `Text.markdown` will toggle Markdown mode being on and off. 
    
2. `Text.data` passes through a `string.Template` operator to replace templated expressions with `IPython.display` `object`s.

    > `Text.format` will toggle the `string.Template` operation.

`Text.source` is extracted from `Text.data` during `Text.__post_init__`.  `Text.source` is block code from the __Markdown___ body; fenced code blocks are executed as magics. 

`Text.expressions` is a `tuple` of inline __Markdown__ code objects.   These expressions will always return a display when there is 
an error to assure that all code objects are reproducible.  `Test.expressions` are motivated by the `IPython.core.interactiveshell.InteractiveShell.user_expressions` that only evaluate single expression

> `Text.inline` toggles the display behavior of `Test.expressions`

In [4]:
    from pidgin.strings.markdown import requote

%reload_ext pidgin.display

In [6]:
    import ast

In [16]:
@dataclass
class Text(collections.UserString):
    __doc__ = """`Text` is an `object` that manages the statements and expressions
    contained within a block of markdown text."""
    
    data: str = None
    
    markdown: bool = True
    format: bool = False
    display: IPython.display.DisplayObject = None
    
    source: str = ''
    
    expressions: tuple = field(default_factory=tuple)
    inline: bool = False
        
    environment: None = field(default=environment)
    template: bool = True
    
    globals: dict = field(default_factory=lambda:IPython.get_ipython().user_ns, repr=False)
        
    ip: IPython.core.interactiveshell.InteractiveShell = field(default=IPython.get_ipython())
    
    html: str = False
    should_display: bool = None
                
    def __post_init__(self):
        # Test if the source is a valid string formatter
        self.raw = self.data
        if self.markdown: 
            self.source, self.expressions = renderer(self.data)
                        
        if self.data and self.format:
            self.data = IPythonDollarTemplate(self.data).safe_substitute(**self.globals)
            
        if not self.markdown: self.source = self.data
            
        self.expressions = set(self.expressions)
        
    def __bool__(self):
        stripped = self.data.strip()
        try:
            if  stripped == ast.literal_eval(self.source).strip(): 
                return False
        except: ...

        if stripped.rstrip().endswith(';'): return False
        if stripped and not self.data.splitlines()[0].strip(): return False
        if textwrap.dedent(self.data).strip() == textwrap.dedent(self.source).strip(): return False
        return True
        
    
    @property
    def _ipython_display_cls_(self): 
        return self.html and IPython.display.HTML or IPython.display.Markdown
    
    def _ipython_display_(self):
        if self:
            self.display = IPython.display.display(self.data, display_id=True)            

In [17]:
class Expression(Text):
    def __iter__(self):
        yield from split_expression(self.data)
        
    def _ipython_display_(self):
        texts = list(self)
        for text in texts:
            exception = None
            try:
                with IPython.utils.capture.capture_output() as capture:
                    object = self(text)
            except BaseException as Exception:
                IPython.display.display(IPython.display.Markdown(F"`>>> {self.data}`")) 
                ip.showtraceback((type(Exception), Exception, Exception.__traceback__))
                break
                
            if self.inline and text is texts[-1]:
                IPython.display.display(IPython.display.Markdown(F"`>>> {self.data}`")) 
                text.strip() and capture.show()
                object and IPython.display.display(object)
                
                
    def __call__(self, source):
        if source.strip():
            return eval(self.ip.input_transformer_manager.transform_cell(source), 
                self.globals, self.globals)
        return None

In [18]:
class Document(Text):    
    def _ipython_display_(self):
        Text._ipython_display_(self)
        for expression in self.expressions:
            IPython.display.display(Expression(expression))

In [19]:
    def split_expression(str, *expressions):
        """Split an expression on the semi colons."""
        start = 0
        if str.startswith(';') and str.strip().strip(';'): 
            """This complies with the special IPython ; syntax"""
            return str, 
        for id in toolz.pipe(
            str, expression_tokens, 
            toolz.filter(toolz.compose(
                ';'.__eq__, operator.attrgetter('string')
            )), 
            toolz.map(toolz.compose(
                toolz.second,
                operator.attrgetter('start')
            )),
            list
        ) + [len(str)]:
            expressions += str[start:id].strip(),
            start = id + 1
        return expressions

In [20]:
    def expression_tokens(str): 
        """Tokenize expressions.
        """
        try: return list(tokenize.generate_tokens(io.StringIO(str).readline))
        except: return []

In [21]:
    def expression_tokens(str): 
        try: return list(tokenize.generate_tokens(io.StringIO(str).readline))
        except: return []